# Phase 3 - Modélisation & Enrichissement Machine Learning

Cette phase exploite directement le fichier `hellowork_preprocessed.csv` issu de ta Phase 2.

## 3.1 Objectifs BI & ML

*   Identifier automatiquement des **groupes de métiers / compétences**
*   Enrichir le dataset avec des **clusters interprétables**
*   Classer les offres selon un **niveau de salaire** exploitable analytiquement
*   Préparer les résultats pour une **visualisation interactive**

## 3.2 Chargement des données

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("hellowork_preprocessed.csv")
# Preserve Date column
if 'Publication_Date' in df.columns:
    df['Publication_Date'] = pd.to_datetime(df['Publication_Date'])

print(df.shape)

## 3.3 Clustering NLP (Descriptions d'offres)

### a) Vectorisation TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=500)
# On s'assure que Description_Clean est bien de type string
X_text = tfidf.fit_transform(df['Description_Clean'].astype(str))

### b) Modélisation par NMF (Non-negative Matrix Factorization)

Nous remplaçons ici KMeans par **NMF**, un algorithme souvent plus performant pour l'extraction de thématiques (Topic Modeling) sur du texte.

In [ ]:
from sklearn.decomposition import NMF

# On choisit 5 thématiques (clusters)
nmf = NMF(n_components=5, random_state=42)

# W contient la matrice Documents x Topics
W = nmf.fit_transform(X_text)

# On assigne chaque offre au topic dominant (celui avec le score le plus élevé)
df['Job_Cluster'] = W.argmax(axis=1)

### c) Interprétation des clusters (Thématiques)

In [ ]:
terms = tfidf.get_feature_names_out()
# Pour NMF, nmf.components_ contient la matrice Topics x Termes
for i in range(5):
    center_terms = nmf.components_[i].argsort()[-10:]
    print(f"Cluster {i}:", [terms[t] for t in center_terms])

## 3.4 Classification - Niveau de salaire

### a) Création de la cible

In [ ]:
median_salary = df['Salary_Clean'].median()
df['High_Salary'] = (df['Salary_Clean'] >= median_salary).astype(int)

### b) Entraînement du modèle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

features = df[['Sector_Encoded', 'Contract_Encoded']].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(
    features, df['High_Salary'], test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test)))

### c) Ajout des prédictions

In [ ]:
df['Salary_Class_Pred'] = model.predict(features)

## 3.5 Sauvegarde dataset enrichi

In [ ]:
df.to_csv("hellowork_ml_enriched.csv", index=False)